# 金融风控特征工程之特征评估与筛选学习目标
- 掌握单特征分析的衡量指标
- 知道IV、PSI等指标含义
- 知道多特征筛选的常用方法
- 掌握Boruta、VIF、RFE、L1特征筛选常见方法

# 1、如何判断一个特征是好特征，从那几个角度衡量？
一般从特征的覆盖度、区分度、相关性、稳定性四个角度分析，确定一个特征的好坏。
- 覆盖度：覆盖度可以衍生出两个指标：缺失率、零值率
- 区分度：是评估一个特征好坏用户的区分性指标
- 相关性：对线性回归模型，有一条基本假设是自变量x1，x2，…，xp之间不存在严格的线性关系
- 稳定性：特征稳定性主要通过计算不同时间段内同一类用户特征的分布的差异来评估

# 2、特征筛选有哪些方法？

- 星座特征：星座是大家公认没用的特征，区分度低于星座的特征可以认为是无用特征

![Boruta算法](./img/Brouta.png)

- Boruta算法：Boruta算法是一种特征选择方法，使用特征的重要性来选取特征
- 方差膨胀系数（Variance Inflation factor VIF），用于检测多重共线性（Multicollinearity）的一个重要统计指标。它在回归分析中被广泛应用，尤其是在多元线性回归中，用于评估自变量之间的线性相关性对回归模型的影响。具体特征如下：
    - 如果一个特征是其他一组特征的线程组合，则不会再模型中提供额外的信息，可以去掉
    - 评价共线性程度：$ \rm{x_i=1+\sum_{k\ne{i}}\beta_{k}x_{k}} $
    - VIF计算：$\rm{VIF=\frac{1}{1-R^2}}$
    - R2是线性会对中的决定系数，反映了回归方程解释因变量变化的百分比，它可以由因变量和自变量之间的复相关系数得到，可以由回归方程的残差平方和和总平方和的比值得到，为了得到每一个变量的VIF，我们需要以每一个变量为因变量对其余所有变量进行线性回归分析，在对每一个变量得到的各自R2，在代入上面的式子，就可以得到每一个变量的VIF
    - VIF越大说明拟合越好，该特征和其他特征组合的共线性越强，就越没有信息量，可以剔除
- 向后筛选:
- L1惩罚项:L1正则化的核心在于其稀疏性特性。由于惩罚项的存在，模型会倾向于将不重要的特征系数压缩到零，而保留重要的特征系数。这种稀疏解的特性使得L1正则化能够自动筛选出对模型预测最有贡献的特征，同时剔除噪声特征
    - 使用L1范数作为惩罚项的线性模型(Linear models)会得到稀疏解：大部分特征对应的系数为0
    - 希望减少特征维度用于其它分类器时，可以通过 feature_selection.SelectFromModel 来选择不为0的系数
- 业务逻辑:
    - 内部特征：特征稳定性、特征区分度、分箱风险区分度
    - 外部特征：覆盖度、区分度、稳定性

# 总结：
- 单特征分析：覆盖度、区分度、稳定性
- 多特征筛选：星座、Boruta、VIF（方差膨胀系数 Variance inflation factor）、RFE（递归特征消除Recursive Feature Elimination）、L1（基于L1的特征选择 L1-based feature selection）、业务
- 内部特征监控：前端稳定性，后段区分度
- 外部特征的评估：评分型数据、名单型数据、保护隐私、未来信息

In [1]:
import pandas as pd

df = pd.DataFrame({
    'A': [5, 91, 3],
    'B': [90, 15, 66],
    'C': [93, 27, 3]
})

df.corr()
df.corr('spearman')
df.corr('kendall')

In [41]:
import pandas as pd
import toad
# !pip3 install toad

data = pd.read_csv('./file/germancredit.csv')
data.replace({'good': 0, 'bad': 1}, inplace=True)
data.shape

In [12]:
data

In [11]:
# 缺失率大于0.5 IV值小于0.05 相关性大于0.7 来进行特征筛选
'''
    target:目标变量列名
    empty:空值剔除比例超过多少的值
    iv:IV值剔除比例小于多少的值
    corr:剔除相关性高于多少的值
    return_drop:返回被剔除的特征列
'''
select_data, drop_list = toad.selection.select(data, target='creditability', empty=0.5, iv=0.05, corr=0.7,
                                               return_drop=True)

print(
    '保留特征：', select_data.shape[1],
    '删除缺失值：', len(drop_list['empty']),
    '低IV删除：', len(drop_list['iv']),
    '高相关删除：', len(drop_list['corr'])
)

In [14]:
import numpy as np
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# !pip3 install boruta

# 加载数据
# pd_data = joblib.load('./file/train_woe.pkl')
# pd_data

# 加载数据（旧版本数据加载）
pd_data = pd.read_pickle('./file/train_woe.pkl')
pd_data

In [15]:
# 处理数据，去掉id和目标值
pd_x = pd_data.drop(['SK_ID_CURR', 'TARGET'], axis=1)

# 特征
x = pd_x.values

# 目标值
y = pd_data[['TARGET']].values
# y = pd_data[['TARGET']].values # 目标
# 将多位数组降低到一维
y = y.ravel()

In [16]:
# 先定义一个随机森林分类器
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
'''
BorutaPy function
estimator : 所使用的分类器
n_estimators : 分类器数量, 默认值 = 1000
max_iter : 最大迭代次数, 默认值 = 100
'''
feat_selector = BorutaPy(rf, n_estimators='auto', random_state=1, max_iter=10)
feat_selector.fit(x, y)

In [20]:
# 计算方差膨胀系数
import numpy as np
import pandas as pd
import joblib

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 加载数据（旧版本数据加载）
pd_data = pd.read_pickle('./file/train_woe.pkl')
#去掉ID和目标值
pd_x = pd_data.drop(['SK_ID_CURR', 'TARGET'], axis=1)

pd_data

In [25]:
# def check_VIF_new(df):
#     list_col = df.columns
#     # 将输入数据转化为矩阵
#     x = np.matrix(df)
#     VIF_list = [variance_inflation_factor(x, i) for i in range(x.shape[1])]
#     VIF = pd.DataFrame({'feature': list_col, 'VIF': VIF_list})
#     max_VIF = max(VIF_list)
#     print(max_VIF)
# df_vif = check_VIF_new(pd_x)
# df_vif

#定义计算函数
def checkVIF_new(df):
    lst_col = df.columns
    x = np.matrix(df)
    VIF_list = [variance_inflation_factor(x,i) for i in range(x.shape[1])]
    VIF = pd.DataFrame({'feature':lst_col,"VIF":VIF_list})
    max_VIF = max(VIF_list)
    print(max_VIF)
    return VIF
df_vif = checkVIF_new(pd_x)
df_vif

In [26]:
# 选取方差膨胀系数>3的特征
df_vif[df_vif['VIF'] > 3]

In [33]:
# 倒入numpy包
import numpy as np
# 导入pandas包
import pandas as pd
# 导入joblib 模型加载包
import joblib

from sklearn.feature_selection import RFE
from sklearn.svm import SVR

# 1、加载数据
# pd_data = joblib.load('./file/final_data.pkl')
# 加载数据（旧版本数据加载）
pd_data = pd.read_pickle('./file/final_data.pkl')
pd_data


In [34]:
# 2、特征提取
pd_x = pd_data.drop(['SK_ID_CURR', 'TARGET'], axis = 1)
x = pd_x.values
# 获取TARGET列数据，通过values将DataFrame转化为NumPy， 通过ravel方法将多维数据展为一维数组
y = pd_data[['TARGET']].values.ravel()

In [37]:
# 3、使用RFE选取特征
'''
RFE（Recursive Feature Elimination）是什么？
RFE，即递归特征消除，是一种特征选择方法，用于从数据集中选择最重要的特征，同时移除不重要的特征。它通过反复训练模型、评估特征重要性并逐步移除最不重要的特征，最终保留对模型性能影响最大的特征子集
'''
# 定义SVR支持向量回归模型
estimator = SVR(kernel = 'linear')
# 初始化RFE并拟合数据，最终保留特征项为3，每次移除不重要的特征为1， 并训练数据
selector = RFE(estimator, n_features_to_select = 3, step=1)
selector = selector.fit(x, y)

dic_ft_select = dict()

ft_list = pd_x.columns.to_list()
selected_arr = selector.support_
for ft, selected in zip(ft_list, selected_arr):
    dic_ft_select[ft] = selected

pd_ft_select = pd.DataFrame({'feature': ft_list , "selected": selected_arr})
pd_ft_select

In [38]:
# 基于L1的特征选择Demo
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

# 加载数据
iris = load_iris()
X, y = iris.data, iris.target

# 使用带L1惩罚项的逻辑回归作为基模型
selector = SelectFromModel(estimator=LogisticRegression(penalty="l1", C=0.1, solver="liblinear"))
X_new = selector.fit_transform(X, y)

# 输出选择的特征
print("原始特征数量：", X.shape[1])
print("选择的特征数量：", X_new.shape[1])

In [39]:
# 基于L1的特征选择 (L1-based feature selection)
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
X.shape

In [40]:
lsvc = LinearSVC(C = 0.01, penalty = 'l1', dual = False).fit(X, y)
model = SelectFromModel(lsvc, prefit = True)
X_new = model.transform(X)
X_new.shape